# HW5 SVM

Original jupyter notebook is uploaded at [GitHub](https://github.com/zhuo34/csmath2022).

## Active set method
\begin{aligned}
\min\quad& f(\mathbf{x}) = \frac{1}{2}\mathbf{x}^TH\mathbf{x}+\mathbf{c}^T\mathbf{x} \\
\textrm{s.t.}\quad& A\mathbf{x} \geq \mathbf{b}
\end{aligned}

In [13]:
import numpy as np
import scipy
import scipy.linalg


def solve_ActiveSet(H, c, A, b, x0=None, epsilon=1e-6):
    def as2darray(a):
        if a.ndim == 1:
            return a[:, np.newaxis]
        return a
        
    def Lagrange(H, c, A, b):
        m, n = A.shape
        AA1 = np.hstack([H, -A.T])
        AA2 = np.hstack([A, np.zeros([m, m])])
        AA = np.vstack([AA1, AA2])
        bb = np.vstack([-c, b])
        xx = scipy.linalg.solve(AA, bb)
        return xx

    m, n = A.shape
    c, b = as2darray(c), as2darray(b)

    if x0 is None:
        # find x0
        idx = np.where(A[0] != 0)[0][0]
        x0 = np.zeros(n)
        x0[idx] = b[idx, 0] / A[0, idx]
    x = as2darray(x0)

    # initialize active set
    active_set = np.where(A @ x == b)[0].tolist()
    print(active_set)
    # return x

    while True:
        m_active = len(active_set)
        output = Lagrange(H, H @ x + c, A[active_set], np.zeros([m_active, 1]))
        d = output[:n]
        print(d)
        if np.sum(np.abs(d)) < epsilon:
            output = Lagrange(H, c, A[active_set], b[active_set])
            lam = np.squeeze(output[n:])
            print(lam)
            return x
            if np.min(lam) >= 0:
                break
            else:
                idx = np.argmin(lam)
                active_set.pop(idx)
        else:
            active_map = np.zeros(m)
            active_map[active_set] = 1
            not_active_map = active_map == 0
            map2 = np.squeeze(A @ d) < 0
            alpha_map = np.logical_and(not_active_map, map2)
            in_active_idx = np.argmin(np.squeeze(A @ d))
            alpha = min(1, np.min((b[alpha_map] - A[alpha_map] @ x) / (A[alpha_map] @ d)))
            
            x += alpha * d

            if alpha != 1:
                active_set.append(in_active_idx)
        break

    return x

H = np.array([[1, 0], [0, 1]])
c = np.array([-2, -5])
A = np.array([
    [1, -2],
    [-1, -2],
    [-1, 2],
    [1, 0],
    [0, 1]
])
b = np.array([-2, -6, -2, 0, 0])
x0 = np.array([2, 0])

x = solve_ActiveSet(H, c, A, b, x0=x0)
print(x)

[2, 4]
[[0.]
 [0.]]
[-0. -5.]
[[2]
 [0]]
